In [2]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [3]:
data = pd.read_csv(r'C:\Users\a_del\Downloads\Customer-Churn.csv')

In [4]:
data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [5]:
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [7]:
data['TotalCharges'].value_counts()

          11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: TotalCharges, Length: 6531, dtype: int64

In [9]:
data['TotalCharges'].isna().sum()#checking for null values 

0

Apply SMOTE for upsampling the data

In [12]:
Xf=data.drop(['Churn'], axis=1)
yf=data['Churn']
Xf_num=Xf.select_dtypes(include = np.number)
Xf_cat = pd.get_dummies(Xf.select_dtypes(include = 'object'), drop_first=True)
Xf=pd.concat([Xf_num, Xf_cat], axis=1)

In [13]:
churndata = pd.concat([Xf,yf],axis=1)
smote = SMOTE()
y = churndata['Churn']
X = churndata.drop(['Churn'], axis=1)
X_sm, y_sm = smote.fit_resample(X, y)

unique, counts = np.unique(y_sm, return_counts=True)
print(np.asarray((unique, counts)).T)

[['No' 5174]
 ['Yes' 5174]]


Use logistic regression to fit the model and compute the accuracy of the model.

In [19]:
Xf=churndata.drop(['Churn'], axis=1)
yf=churndata['Churn']
    
X_train, X_test, y_train, y_test = train_test_split(Xf, yf, test_size=0.3, random_state=42)

In [21]:
model = LogisticRegression(max_iter = 500)
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

C:\Users\a_del\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

          No       0.85      0.90      0.87      1539
         Yes       0.68      0.56      0.61       574

    accuracy                           0.81      2113
   macro avg       0.76      0.73      0.74      2113
weighted avg       0.80      0.81      0.80      2113



Use decision tree classifier to fit the model and compute the accuracy of the model.

In [25]:
model = DecisionTreeClassifier(random_state=0)
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          No       0.82      0.88      0.85      1539
         Yes       0.60      0.47      0.53       574

    accuracy                           0.77      2113
   macro avg       0.71      0.68      0.69      2113
weighted avg       0.76      0.77      0.76      2113



Compare the accuracies of the two models.

We have higher scores overall with the logistic regression


Apply TomekLinks for downsampling

In [36]:
tl = TomekLinks()
X_tl, y_tl = tl.fit_resample(Xf, yf)


unique, counts = np.unique(y_tl, return_counts=True)
print(np.asarray((unique, counts)).T)

[['No' 4716]
 ['Yes' 1869]]


Use logistic regression to fit the model and compute the accuracy of the model.

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_tl, y_tl, test_size=0.3, random_state=42)

In [38]:
model = LogisticRegression(max_iter = 500)
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

C:\Users\a_del\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

          No       0.85      0.89      0.87      1410
         Yes       0.70      0.61      0.65       566

    accuracy                           0.81      1976
   macro avg       0.78      0.75      0.76      1976
weighted avg       0.81      0.81      0.81      1976



Use decision tree classifier to fit the model and compute the accuracy of the model.

In [39]:
model = DecisionTreeClassifier(random_state=0)
model.fit( X_train, y_train )
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          No       0.83      0.88      0.86      1410
         Yes       0.66      0.55      0.60       566

    accuracy                           0.79      1976
   macro avg       0.74      0.72      0.73      1976
weighted avg       0.78      0.79      0.78      1976



Compare the accuracies of the two models.

The logistic regression seems better overall as previously 